In [1]:
#!conda install nltk

In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
messages = [line.rstrip() for line in open('C:/Users/NITHISH PATEL/SMSSpamCollection')]
print(len(messages))

5574


In [3]:
for message_no,message in enumerate(messages[:10]):
    print(message_no,message)
    print('\n')

0 ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


1 ham	Ok lar... Joking wif u oni...


2 spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


3 ham	U dun say so early hor... U c already then say...


4 ham	Nah I don't think he goes to usf, he lives around here though


5 spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, Â£1.50 to rcv


6 ham	Even my brother is not like to speak with me. They treat me like aids patent.


7 ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune


8 spam	WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim cod

In [ ]:
import pandas as pd

In [ ]:
messages=pd.read_csv('C:/Users/NITHISH PATEL/SMSSpamCollection',sep='\t',
                     names=["label","messages"])

In [ ]:
messages.head()

In [ ]:
messages.groupby('label').describe()

In [ ]:
messages.describe()

In [ ]:
messages['length'] = messages['messages'].apply(len)
messages.head()

In [ ]:

import matplotlib.pylot as pl
%matplotlib inline

In [ ]:
messages['length'].plot(bins=50,kind='hist')

In [ ]:
messages.length.describe()

In [ ]:
messages[messages['length']==910]['messages'].iloc[0]

In [ ]:
messages.hist*****

In [ ]:
import string
mess='sample message! Notice: it has panctuation.'
#check characters 
nopunc=[char for char in mess if char not in string.punctuation]
#join the characters to form string
nopunc=''.join(nopunc)

In [ ]:
from nltk.corpus import stopwords
stopwords.words('english')[0:20]#show some 

In [ ]:
nopunc.split()

In [ ]:
#now remove stop words
clean_mess=[word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
clean_mess

In [ ]:
def text_process(mess):
    nopunc=[char for char in mess if char not in string.punctuation]
    nopunc=''.join(nopunc)
    return[word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
messages['messages'].head(5).apply(text_process)

In [ ]:
messages.head()

In [ ]:
Normalization

In [ ]:
VECTORIZATION

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(messages['messages'])

print(len(bow_transformer.vocabulary_))

In [ ]:
message4=messages['messages'][3]
print(message4)

In [ ]:
bow4 = bow_transformer.transform([messages])
print(bow4)
print(bow4.shape)

In [ ]:
print(bow_transformer.get_feature_names()[4073])
print(bow_transformer.get_feature_names()[9570])

In [ ]:
messages_bow=bow_transformer.transform(messages['messages'])

In [ ]:
print('shape of sparse matrix')messages_bow.shape
print('amount of non zero occurance')messages_bow.nnz

In [ ]:
sparsity=(100.0*messages_bow.nnz/(messages_bow.shape[0]*messages_bow.shape[1]))
print('sparsity:{}'.format(round(sparsity)))

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer().fit(messages_bow)
tfidf4=tfidf_transformer.transform(bow4)
print(tfidf4)

In [ ]:
messages_tfidf=tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model=MultinomialNB().fit(messages_tfidf,messages['label'])

In [ ]:
print('predicted:',spam_detect_model.predict(tfidf4)[0])
print('expected:',messages.label[3])

In [ ]:
#model evaluation

In [ ]:
all_predictions=spam_detect_model.predict(messages_tfidf)
all_predictions

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(messages['label'],all_predictions))

In [ ]:
from sklearn.model_selection import train_test_split
msg_train,msg_test,label_train,label_test=\
train_test_split(messages['messages'],messages['label'],test_size=0.2)

In [ ]:
from sklearn.pipeline import Pipeline
pipeline=Pipeline([
                  ('bow',CountVectorizer(analyzer=text_process)),
                  ('tfidf',TfidfTransformer()),
                  ('classifier',MultinomialNB()),
                  ])

In [ ]:
pipeline.fit(msg_train,label_train)

In [ ]:
predictions=pipeline.predict(msg_test)

In [ ]:
print(classification_report(predictions,label_test))